# 题目：
## 设计一个类，我们只能生成该类的一个实例
### 闭包模式&元类模式

参考：http://www.cnblogs.com/yssjun/p/9858420.html

### 1. 闭包模式
先理解python闭包概念：

先看一个例子：

In [7]:
#outer_func内嵌inner_function，注意到inner_func外部函数的返回值
#inner_function也称为闭包函数，outer_function称为闭包函数引用的父函数
#称loc_list为闭包函数的一个自由变量，这一local variable既不具有C++中的全局变量的性质也没有static变量的行为
def outer_func():
     loc_list = []
     def inner_func(name):
         loc_list.append(len(loc_list) + 1)
         print ('%s loc_list = %s' %(name, loc_list))
     return inner_func

clo_func_0 = outer_func()
clo_func_0('clo_func_0')
clo_func_0('clo_func_0')
clo_func_0('clo_func_0')

clo_func_1 = outer_func()
clo_func_1('clo_func_1')
clo_func_0('clo_func_0')
clo_func_1()

clo_func_0 loc_list = [1]
clo_func_0 loc_list = [1, 2]
clo_func_0 loc_list = [1, 2, 3]
clo_func_1 loc_list = [1]
clo_func_0 loc_list = [1, 2, 3, 4]


在这个例子中我们至少可以对闭包中引用的自由变量有如下的认识：

①闭包中的引用的自由变量只和具体的闭包有关联，闭包的每个实例引用的自由变量互不干扰。

    -比如说clo_func_0和clo_func_1两个的闭包函数不同，所以闭包引用的自由变量互不干扰

②一个闭包实例对其自由变量的修改会被传递到下一次该闭包实例的调用。

值得注意的一条经验！

    - 返回闭包中不要引用任何循环变量，或者后续会发生变化的变量。

In [12]:
def singleton(cls, *args, **kwargs):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class my_cls(object):
    pass

t1 = my_cls()
t2 = my_cls()

print(t1,t2)
#t1，t2是同个类，这就实现了单类模式

<__main__.my_cls object at 0x0000020FF8D48EB8> <__main__.my_cls object at 0x0000020FF8D48EB8>


### 2. 元类模式
所谓单例模式，即我们需要控制类实例的生成过程，并且保证全局只可能存在一个唯一的实例。

既然需要在创建类的对象过程中做些什么，应该很容易想到元类metaclass。

In [13]:
class Singleton(type):
    def __init__(cls, name, bases, dic):
        super(Singleton, cls).__init__(names, bases, dic)
        cls._instance = None
    
    def __call__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instance
    
class my_cls(object):
    __metaclass__ = Singleton

这个例子中我们使用元类Singleton替代默认使用type方式创建类my_cls。可以将类my_cls看做是元类Singleton的一个对象，当我们使用my_cls(...)的方式创建类my_cls的对象时，实际上是在调用元类Singleton的对象my_cls。

对象可以以函数的方式被调用，那么要求类中定义\__call\__函数。不过此处被调用的是类，因此我们在元类中定义函数__call__来控制类my_cls对象创建的唯一性。

这种方式的弊端之一就是类唯一的对象被存放在类的一个静态数据成员中，外部可以通过class_name._instance的方式修改甚至删除这个实例(该例中my_cls._instance = None完全合法)。